<a href="https://colab.research.google.com/github/happyhillll/DigitalHumanities/blob/main/%ED%95%9C%EA%B5%AD%EA%B3%A0%EC%A0%84%EC%A2%85%ED%95%A9DB_XML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한국고전종합DB XML
한국문집총간 XML : https://www.data.go.kr/data/3074298/fileData.do  
고전원문 XML : https://www.data.go.kr/data/15022432/fileData.do


In [1]:
pip install -U  -q gdown natsort

In [2]:
import pandas as pd
import numpy as np
import lxml
import xml.etree.ElementTree as ET
from tqdm import tqdm
import numpy as np
import re
from natsort import natsorted
import glob
import gdown

In [3]:
# 한국문집총간 다운 (총 1259개 문서)
# https://drive.google.com/file/d/1GOER7ac1U92_yRT-2bKSFlZRpZQQpN8-/view?usp=sharing
!gdown 1GOER7ac1U92_yRT-2bKSFlZRpZQQpN8-

Downloading...
From: https://drive.google.com/uc?id=1GOER7ac1U92_yRT-2bKSFlZRpZQQpN8-
To: /content/한국문집총간.zip
100% 447M/447M [00:01<00:00, 286MB/s]


In [4]:
# 1차 압축해제
!unzip '한국문집총간.zip'

Archive:  한국문집총간.zip
   creating: zip/
  inflating: zip/1. 계원필경집(桂苑筆耕集).zip  
  inflating: zip/10. 근재집(謹齋集).zip  
  inflating: zip/100. 십청집(十淸集).zip  
  inflating: zip/1000. 백남집(白南集).zip  
  inflating: zip/1001. 퇴어당유고(退漁堂遺稿).zip  
  inflating: zip/1002. 봉암집(鳳巖集).zip  
  inflating: zip/1003. 노은집(老隱集).zip  
  inflating: zip/1004. 둔암집(屯菴集).zip  
  inflating: zip/1005. 매산집(梅山集).zip  
  inflating: zip/1006. 관아재고(觀我齋稿).zip  
  inflating: zip/1007. 목곡집(牧谷集).zip  
  inflating: zip/1008. 관수재유고(觀水齋遺稿).zip  
  inflating: zip/1009. 모헌집(慕軒集).zip  
  inflating: zip/101. 안분당시집(安分堂詩集).zip  
  inflating: zip/1010. 화계집(花溪集).zip  
  inflating: zip/1011. 동계유고(東溪遺稿).zip  
  inflating: zip/1012. 서재집(西齋集).zip  
  inflating: zip/1013. 퇴헌유고(退軒遺稿).zip  
  inflating: zip/1014. 황고집(黃臯集).zip  
  inflating: zip/1015. 석문집(石門集).zip  
  inflating: zip/1016. 국포집(菊圃集).zip  
  inflating: zip/1017. 일암집(一菴集).zip  
  inflating: zip/1018. 이치재집(二恥齋集).zip  
  inflating: zip/1019. 한송재집(寒松齋集).zip  
  inflating: zip/102. 우암고(寓菴稿)

In [6]:
# 2차 압축해제(압축파일 속 압축파일까지 압축해제)
!find . -name '*.zip' -exec sh -c 'unzip -d "${1%.*}" "$1"' _ {} \;

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: ./한국문집총간/zip/597. 여유당전서(與猶堂全書)/ITKC_MO_0597A_0820.xml  
  inflating: ./한국문집총간/zip/597. 여유당전서(與猶堂全書)/ITKC_MO_0597A_0830.xml  
  inflating: ./한국문집총간/zip/597. 여유당전서(與猶堂全書)/ITKC_MO_0597A_0840.xml  
  inflating: ./한국문집총간/zip/597. 여유당전서(與猶堂全書)/ITKC_MO_0597A_0850.xml  
  inflating: ./한국문집총간/zip/597. 여유당전서(與猶堂全書)/ITKC_MO_0597A_0860.xml  
  inflating: ./한국문집총간/zip/597. 여유당전서(與猶堂全書)/ITKC_MO_0597A_0870.xml  
  inflating: ./한국문집총간/zip/597. 여유당전서(與猶堂全書)/ITKC_MO_0597A_0880.xml  
  inflating: ./한국문집총간/zip/597. 여유당전서(與猶堂全書)/ITKC_MO_0597A_0890.xml  
  inflating: ./한국문집총간/zip/597. 여유당전서(與猶堂全書)/ITKC_MO_0597A_0900.xml  
  inflating: ./한국문집총간/zip/597. 여유당전서(與猶堂全書)/ITKC_MO_0597A_0910.xml  
  inflating: ./한국문집총간/zip/597. 여유당전서(與猶堂全書)/ITKC_MO_0597A_0920.xml  
  inflating: ./한국문집총간/zip/597. 여유당전서(與猶堂全書)/ITKC_MO_0597A_0930.xml  
  inflating: ./한국문집총간/zip/597. 여유당전서(與猶堂全書)/ITKC_MO_0597A_0940.xml  
  inflating: ./한국문집총간/zip/597. 여유당전서(與猶堂全書)/ITKC_MO_0597A_0950.xml 

In [ ]:
N!for i in ./zip/*.zip; do mkdir  ${i%%.zip}; unzip "$i" -d "${i%%.zip}"; done

Archive:  ./zip/1000. 백남집(白南集).zip
replace ./zip/1000. 백남집(白南集)/ITKC_MO_1000A.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  ./zip/1001. 퇴어당유고(退漁堂遺稿).zip
replace ./zip/1001. 퇴어당유고(退漁堂遺稿)/ITKC_MO_1001A.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  ./zip/1002. 봉암집(鳳巖集).zip
replace ./zip/1002. 봉암집(鳳巖集)/ITKC_MO_1002A.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  ./zip/1003. 노은집(老隱集).zip
replace ./zip/1003. 노은집(老隱集)/ITKC_MO_1003A.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  ./zip/1004. 둔암집(屯菴集).zip
replace ./zip/1004. 둔암집(屯菴集)/ITKC_MO_1004A.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  ./zip/1005. 매산집(梅山集).zip
replace ./zip/1005. 매산집(梅山集)/ITKC_MO_1005A.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  ./zip/1006. 관아재고(觀我齋稿).zip
replace ./zip/1006. 관아재고(觀我齋稿)/ITKC_MO_1006A.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  ./zip/1007. 목곡집(牧谷集).zip
replace ./zip/1007. 목곡집(牧谷集)/ITKC_MO_1007A.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Arch

### 파일 목록 정리

In [ ]:
file_list = natsorted(glob.glob('./zip/*/*.xml'))

In [ ]:
print(len(file_list))
print(len(file_list)-1259) # 텍스트 존재 XML 파일 수

18392
17133


In [ ]:
# 텍스트 있는 xml 파일만 추출
file_list_text = [f for f in file_list if re.match('ITKC\_MO\_.+\_\d+\.xml',f.split('/')[-1])]
len(file_list_text)

17133

In [ ]:
file_list_text[0].split('/')[-2] # 서명

'1. 계원필경집(桂苑筆耕集)'

In [ ]:
file_list_text[1].split('/')[-1] # 파일명

'ITKC_MO_0001A_0020.xml'

### 파싱 테스트

In [ ]:
tree = ET.parse(file_list_text[0])

In [ ]:
tree.find('.//레벨1').attrib['id']

'ITKC_MO_0001A'

In [ ]:
tree.find('.//레벨4').attrib['DCI']

'ITKC_MO_0001A_0010_010_0010_2003_A001_XML'

In [ ]:
# 원문 id
tree.find('.//레벨4').attrib['id']

'ITKC_MO_0001A_0010_010_0010'

In [ ]:
# 연계항목(이미지, 번역, 교감표점) id
tree.findall('.//레벨4//연계항목')

[<Element '연계항목' at 0x7fbdf66d0450>,
 <Element '연계항목' at 0x7fbdf66d0540>,
 <Element '연계항목' at 0x7fbdf66c81d0>,
 <Element '연계항목' at 0x7fbdf66c82c0>,
 <Element '연계항목' at 0x7fbdf66cb9a0>,
 <Element '연계항목' at 0x7fbdf66cba90>]

In [ ]:
tree.findall('.//레벨4//연계항목')[0].attrib['type']

'이미지'

In [ ]:
tree.findall('.//레벨4//연계항목')[1].attrib['type']

'번역문'

In [ ]:
tree.findall('.//레벨4//연계항목')[1].attrib['연계시작']

'ITKC_BT_0001A_0010_000_0010'

In [ ]:
tree.find('.//레벨4//제목정보')

<Element '제목정보' at 0x7fbdf66d8a90>

In [ ]:
tree.findall('.//레벨4//내용')

[<Element '내용' at 0x7fbdf66d8f40>,
 <Element '내용' at 0x7fbdf66d0bd0>,
 <Element '내용' at 0x7fbdf66c8950>]

In [ ]:
tree.findall('.//레벨4//내용')[0].find('단락').text

'記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本'

In [ ]:
tree.findall('.//레벨4//내용')[0].findtext('단락')

'記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本'

In [ ]:
tree.findall('.//레벨4//내용')[0].attrib

{}

In [ ]:
tree.findall('.//레벨4//내용')[0].tag

'내용'

In [ ]:
for para in tree.iter('단락'):
  print(para.text)

記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本
None
None
私試今體賦五首一卷
右臣自年十二。離家西泛。當乘桴之際。亡父誡之曰。
中和六年正月日。前
None
None


In [ ]:
tree.findall('.//레벨4//내용')[0].find('단락').itertext()

In [ ]:
''.join(tree.findall('.//레벨4//내용')[0].find('단락').itertext())

'記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本\n也。黼黻文繡之美。而疏布之尙。反女功之始也。古之\n君子。必重其本始如此。吾東方之有文章而能著書\n傳後者。自孤雲崔公始。吾東方之士。北學于中國。而\n以文聲天下者。亦自崔公始。崔公之書傳于後者。唯\n桂苑筆耕與中山覆簣集二部。是二書者。亦吾東方\n文章之本始也。吾東方以文爲尙。至我朝益煥以\n融。家燕許而戶曹劉。以詩若文成集者。無慮充棟宇\n矣。而顧鮮有知崔公之書者。余嘗見近代人所撰東\n國書目。有載中山覆簣集者。徧求之。終不可得。唯桂\n苑筆耕二十卷。爲吾家先世舊藏。自童幼時。知珍而\n玩之。然間以語人。雖博雅能文而好古者。亦皆言未\n曾見。然則是書也幾乎絶矣。使是書不行于東方。是\n玄酒不設于太室。而疏布不羃于犠罇也。豈所以敎\n民不忘本哉。世或謂公文皆騈儷四六。殊不類古作\n者。公之入中國。在唐懿僖之際。中國之文。方專事騈\n儷。風會所趨。固有不得而免者。然觀公所爲辭。往往\n多華而不浮。如檄黃巢一篇。氣勁意直。絶不以雕鏤\n爲工。至其詩平易近雅。尤非晩唐人所可及。是蓋以\n明水疏布之質。而兼有乎酒醴黼黻之美者。豈不彌\n可珍哉。公在中國。取科第入軍府。亦旣已聲施當時\n矣。而一朝去之如脫屣。及歸東方。躋翰苑。貳兵部。以\n至阿飡。阿飡者。新羅大官。其顯用方未已也。而顧又\n自放於山林寂寞之濱。以終老其身而不悔。蓋度其\n時之皆不可有爲也。士君子立身蹈道。莫有大乎出\n處之際。出處而不失其時。非賢者。不能也。賢者之作。\n固不可使其無傳。況其文傑然如彼。而又爲東國文\n章之本始者哉。湖南觀察使徐公準平。卽余所稱博\n雅能文而好古者也。聞余蓄是書。亟取而校之。捐其\n俸搨以活字。得數十百本。用廣其傳曰。不可使是書\n絶于東國也。嗚呼。不忘本始。敎民厚也。表章賢人。勸\n民善也。徐公之用心也如此。其所以爲政於湖南者。\n亦可知已。役旣完。徐公屬余曰。子實傳是書。今不可\n以靳一言。余辭不能得。若崔公之蹟行本末與是書\n之可備攷證者。徐公之序詳之矣。余無所復贅云。甲\n午九月。大匡輔國崇祿大夫議政府左議政豐山洪\n奭周。序。'

In [ ]:
re.sub('\n','',''.join(tree.findall('.//레벨4//내용')[0].find('단락').itertext()))

'記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古之君子。必重其本始如此。吾東方之有文章而能著書傳後者。自孤雲崔公始。吾東方之士。北學于中國。而以文聲天下者。亦自崔公始。崔公之書傳于後者。唯桂苑筆耕與中山覆簣集二部。是二書者。亦吾東方文章之本始也。吾東方以文爲尙。至我朝益煥以融。家燕許而戶曹劉。以詩若文成集者。無慮充棟宇矣。而顧鮮有知崔公之書者。余嘗見近代人所撰東國書目。有載中山覆簣集者。徧求之。終不可得。唯桂苑筆耕二十卷。爲吾家先世舊藏。自童幼時。知珍而玩之。然間以語人。雖博雅能文而好古者。亦皆言未曾見。然則是書也幾乎絶矣。使是書不行于東方。是玄酒不設于太室。而疏布不羃于犠罇也。豈所以敎民不忘本哉。世或謂公文皆騈儷四六。殊不類古作者。公之入中國。在唐懿僖之際。中國之文。方專事騈儷。風會所趨。固有不得而免者。然觀公所爲辭。往往多華而不浮。如檄黃巢一篇。氣勁意直。絶不以雕鏤爲工。至其詩平易近雅。尤非晩唐人所可及。是蓋以明水疏布之質。而兼有乎酒醴黼黻之美者。豈不彌可珍哉。公在中國。取科第入軍府。亦旣已聲施當時矣。而一朝去之如脫屣。及歸東方。躋翰苑。貳兵部。以至阿飡。阿飡者。新羅大官。其顯用方未已也。而顧又自放於山林寂寞之濱。以終老其身而不悔。蓋度其時之皆不可有爲也。士君子立身蹈道。莫有大乎出處之際。出處而不失其時。非賢者。不能也。賢者之作。固不可使其無傳。況其文傑然如彼。而又爲東國文章之本始者哉。湖南觀察使徐公準平。卽余所稱博雅能文而好古者也。聞余蓄是書。亟取而校之。捐其俸搨以活字。得數十百本。用廣其傳曰。不可使是書絶于東國也。嗚呼。不忘本始。敎民厚也。表章賢人。勸民善也。徐公之用心也如此。其所以爲政於湖南者。亦可知已。役旣完。徐公屬余曰。子實傳是書。今不可以靳一言。余辭不能得。若崔公之蹟行本末與是書之可備攷證者。徐公之序詳之矣。余無所復贅云。甲午九月。大匡輔國崇祿大夫議政府左議政豐山洪奭周。序。'

### 실전 XML 파싱

In [ ]:
row_base_list = []
row_asso_list = []
for xml in tqdm(file_list_text):
    tree = ET.parse(xml)
    
    # 서지ID
    bib = tree.find('.//레벨1').attrib['id']
    
    # 서명
    book = xml.split('/')[-2] # 서명

    if tree.findall('.//레벨4')!=[]:
        # 기본정보
        lv4_list = tree.findall('.//레벨4')

        # 내용 부분
        lv4_content_list = tree.findall('.//레벨4//본문정보')

        # 연계항목
        # lv4_asso_list = tree.findall('.//레벨4//연계항목')
        
    else: #레벨3에 위치한 경우
        # print('lv3')
        # 기본정보
        lv4_list = tree.findall('.//레벨3')

        # 내용 부분
        lv4_content_list = tree.findall('.//레벨3//본문정보')

        # 연계항목
        # lv4_asso_list = tree.findall('.//레벨3//연계항목')


    # 자료ID
    id = [lv4.attrib['id'] for lv4 in lv4_list]

    # DCI_s
    dci = [lv4.attrib['DCI'] for lv4 in lv4_list]

    ## 내용
    content = [''.join(lv4_content.find('내용').itertext()).strip() for lv4_content in lv4_content_list]

    
    # ## 번역문ID
    # transID = [(lv4_asso.attrib['연계시작'],lv4_asso.attrib['연계종료']) for lv4_asso in lv4_asso_list if lv4_asso.attrib['type']=='번역문']
    # ## 교감표점ID
    # kyoID = [(lv4_asso.attrib['연계시작'],lv4_asso.attrib['연계종료']) for lv4_asso in lv4_asso_list if lv4_asso.attrib['type']=='교감표점']
    # ## 이미지ID
    # imageID = [(lv4_asso.attrib['연계시작'],lv4_asso.attrib['연계종료']) for lv4_asso in lv4_asso_list if lv4_asso.attrib['type']=='이미지']

    # 연계정보
    res_asso = [{'id':lv4.attrib['id'],'type':asso.attrib['type'], '연계시작':asso.attrib['연계시작'],'연계종료':asso.attrib['연계종료']} for lv4 in lv4_list for asso in lv4.findall('.//연계항목')]
        
    # dict로 합치기
    # res = {'bib':bib, 'book':book, 'id':id, 'dci':dci, 'content':content, 'transID':transID, 'kyoID':kyoID, 'imageID':imageID}
    res_base = {'bib':bib, 'book':book, 'id':id, 'dci':dci, 'content':content}

    row_base_list.append(res_base)
    row_asso_list.extend(res_asso)


100%|██████████| 17133/17133 [03:20<00:00, 85.48it/s]


##### 기본 정보

In [ ]:
# 본문 xml 문서수 17713행 확인
df = pd.DataFrame(row_base_list)
df

,bib,book,id,dci,content
0,ITKC_MO_0001A,1. 계원필경집(桂苑筆耕集),"[ITKC_MO_0001A_0010_010_0010, ITKC_MO_0001A_00...","[ITKC_MO_0001A_0010_010_0010_2003_A001_XML, IT...",[記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本\n也。黼黻文繡之美。而疏布之尙。反女功之始...
1,ITKC_MO_0001A,1. 계원필경집(桂苑筆耕集),"[ITKC_MO_0001A_0020_010_0010, ITKC_MO_0001A_00...","[ITKC_MO_0001A_0020_010_0010_2003_A001_XML, IT...",[臣某言。今月某日。得進奏院狀報。奉十一日宣下改\n廣明元年爲中和元年者。展義龜城。易名鳳紀...
2,ITKC_MO_0001A,1. 계원필경집(桂苑筆耕集),"[ITKC_MO_0001A_0030_010_0010, ITKC_MO_0001A_00...","[ITKC_MO_0001A_0030_010_0010_2003_A001_XML, IT...",[臣某言。今月某日。宣慰使供奉官嚴遵美至。奉宣聖\n旨慰諭臣及將校等。並賜臣勑書手詔各一封。...
3,ITKC_MO_0001A,1. 계원필경집(桂苑筆耕集),"[ITKC_MO_0001A_0040_010_0010, ITKC_MO_0001A_00...","[ITKC_MO_0001A_0040_010_0010_2003_A001_XML, IT...",[右臣伏奉四月十日詔旨。黃巢兇逆。穢黷宮城。罪惡\n貫盈。人神共怒。尋東兵合勢。翦滅元兇。想...
4,ITKC_MO_0001A,1. 계원필경집(桂苑筆耕集),"[ITKC_MO_0001A_0050_010_0010, ITKC_MO_0001A_00...","[ITKC_MO_0001A_0050_010_0010_2003_A001_XML, IT...",[營田判官將仕郞殿中侍御史內供奉賜緋\n魚袋宋絢\n右件官。相門傳慶。詞苑成名。退居安東郭之...
...,...,...,...,...,...
17128,ITKC_MO_1259A,1259. 소눌집(小訥集),"[ITKC_MO_1259A_0440_010_0010, ITKC_MO_1259A_04...","[ITKC_MO_1259A_0440_010_0010_2021_B150_XML, IT...",[先生諱大有字天祐。三足堂其燕居也。金氏駕洛國\n首露王之裔也。因以金海爲貫。版圖判書管，經...
17129,ITKC_MO_1259A,1259. 소눌집(小訥集),"[ITKC_MO_1259A_0450_010_0010, ITKC_MO_1259A_04...","[ITKC_MO_1259A_0450_010_0010_2021_B150_XML, IT...",[國之衰。賢人在野。不能設施於時。然求有以風勵一\n方者則必於斯人焉。古琴孫公。讀書論道七十...
17130,ITKC_MO_1259A,1259. 소눌집(小訥集),"[ITKC_MO_1259A_0460_010_0010, ITKC_MO_1259A_04...","[ITKC_MO_1259A_0460_010_0010_2021_B150_XML, IT...",[公諱奎燦字贊吉號愚潭。朴氏順天人。麗季有平陽\n府院君諱天祥。與圃隱鄭先生同日取義。生諱可...
17131,ITKC_MO_1259A,1259. 소눌집(小訥集),[ITKC_MO_1259A_0470_010_0010],[ITKC_MO_1259A_0470_010_0010_2021_B150_XML],[正宗莊孝大王二十一年丁巳。\n十二月甲子。二十九日卯時 先生生于抱川縣木洞里第。\n許氏本...


In [ ]:
# id가 비어있는 행 제외
print(df[df['id'].map(lambda x:len(x)==0)])
df = df[df['id'].map(lambda x:len(x)!=0)]

                 bib               book  id dci content
2113   ITKC_MO_0203A      203. 송강집(松江集)  []  []      []
7060   ITKC_MO_0486A      486. 여호집(黎湖集)  []  []      []
7061   ITKC_MO_0486A      486. 여호집(黎湖集)  []  []      []
8742   ITKC_MO_0576A    576. 무명자집(無名子集)  []  []      []
8767   ITKC_MO_0577A  577. 청장관전서(靑莊館全書)  []  []      []
8768   ITKC_MO_0577A  577. 청장관전서(靑莊館全書)  []  []      []
8771   ITKC_MO_0577A  577. 청장관전서(靑莊館全書)  []  []      []
8772   ITKC_MO_0577A  577. 청장관전서(靑莊館全書)  []  []      []
10007  ITKC_MO_0617A      617. 오서집(梧墅集)  []  []      []
10012  ITKC_MO_0617A      617. 오서집(梧墅集)  []  []      []
10013  ITKC_MO_0617A      617. 오서집(梧墅集)  []  []      []
10022  ITKC_MO_0617A      617. 오서집(梧墅集)  []  []      []


In [ ]:
# id
df['id'][0]

['ITKC_MO_0001A_0010_010_0010',
 'ITKC_MO_0001A_0010_010_0020',
 'ITKC_MO_0001A_0010_010_0030']

In [ ]:
# dci
df['dci'][0]

['ITKC_MO_0001A_0010_010_0010_2003_A001_XML',
 'ITKC_MO_0001A_0010_010_0020_2003_A001_XML',
 'ITKC_MO_0001A_0010_010_0030_2003_A001_XML']

In [ ]:
# 계원필경 첫파일의 첫 본문 확인
df['content'][0]

['記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本\n也。黼黻文繡之美。而疏布之尙。反女功之始也。古之\n君子。必重其本始如此。吾東方之有文章而能著書\n傳後者。自孤雲崔公始。吾東方之士。北學于中國。而\n以文聲天下者。亦自崔公始。崔公之書傳于後者。唯\n桂苑筆耕與中山覆簣集二部。是二書者。亦吾東方\n文章之本始也。吾東方以文爲尙。至我朝益煥以\n融。家燕許而戶曹劉。以詩若文成集者。無慮充棟宇\n矣。而顧鮮有知崔公之書者。余嘗見近代人所撰東\n國書目。有載中山覆簣集者。徧求之。終不可得。唯桂\n苑筆耕二十卷。爲吾家先世舊藏。自童幼時。知珍而\n玩之。然間以語人。雖博雅能文而好古者。亦皆言未\n曾見。然則是書也幾乎絶矣。使是書不行于東方。是\n玄酒不設于太室。而疏布不羃于犠罇也。豈所以敎\n民不忘本哉。世或謂公文皆騈儷四六。殊不類古作\n者。公之入中國。在唐懿僖之際。中國之文。方專事騈\n儷。風會所趨。固有不得而免者。然觀公所爲辭。往往\n多華而不浮。如檄黃巢一篇。氣勁意直。絶不以雕鏤\n爲工。至其詩平易近雅。尤非晩唐人所可及。是蓋以\n明水疏布之質。而兼有乎酒醴黼黻之美者。豈不彌\n可珍哉。公在中國。取科第入軍府。亦旣已聲施當時\n矣。而一朝去之如脫屣。及歸東方。躋翰苑。貳兵部。以\n至阿飡。阿飡者。新羅大官。其顯用方未已也。而顧又\n自放於山林寂寞之濱。以終老其身而不悔。蓋度其\n時之皆不可有爲也。士君子立身蹈道。莫有大乎出\n處之際。出處而不失其時。非賢者。不能也。賢者之作。\n固不可使其無傳。況其文傑然如彼。而又爲東國文\n章之本始者哉。湖南觀察使徐公準平。卽余所稱博\n雅能文而好古者也。聞余蓄是書。亟取而校之。捐其\n俸搨以活字。得數十百本。用廣其傳曰。不可使是書\n絶于東國也。嗚呼。不忘本始。敎民厚也。表章賢人。勸\n民善也。徐公之用心也如此。其所以爲政於湖南者。\n亦可知已。役旣完。徐公屬余曰。子實傳是書。今不可\n以靳一言。余辭不能得。若崔公之蹟行本末與是書\n之可備攷證者。徐公之序詳之矣。余無所復贅云。甲\n午九月。大匡輔國崇祿大夫議政府左議政豐山洪\n奭周。序。',
 '桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府\n時公私應酬之作。而東還之後。手編表進于朝者也\n公名致遠。字海夫。孤雲其號也。湖南之沃溝人。幼穎\n慧絶倫。年十二。從商舶入中原。十八

In [ ]:
# 본문(content) 리스트로 묶여 있기 때문에 explode 기능으로 풀어주기
df_explode = df.explode(df.columns[2:].tolist(),ignore_index=True) # 세번째 열부터(id, dci, content)
df_explode

,bib,book,id,dci,content
0,ITKC_MO_0001A,1. 계원필경집(桂苑筆耕集),ITKC_MO_0001A_0010_010_0010,ITKC_MO_0001A_0010_010_0010_2003_A001_XML,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本\n也。黼黻文繡之美。而疏布之尙。反女功之始也...
1,ITKC_MO_0001A,1. 계원필경집(桂苑筆耕集),ITKC_MO_0001A_0010_010_0020,ITKC_MO_0001A_0010_010_0020_2003_A001_XML,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府\n時公私應酬之作。而東還之後。手編表進于朝者也\...
2,ITKC_MO_0001A,1. 계원필경집(桂苑筆耕集),ITKC_MO_0001A_0010_010_0030,ITKC_MO_0001A_0010_010_0030_2003_A001_XML,淮南入本國兼送詔書等使前都統巡官承務郎\n侍御史內供奉賜紫金魚袋臣崔致遠進所著雜\n詩賦及表...
3,ITKC_MO_0001A,1. 계원필경집(桂苑筆耕集),ITKC_MO_0001A_0020_010_0010,ITKC_MO_0001A_0020_010_0010_2003_A001_XML,臣某言。今月某日。得進奏院狀報。奉十一日宣下改\n廣明元年爲中和元年者。展義龜城。易名鳳紀。...
4,ITKC_MO_0001A,1. 계원필경집(桂苑筆耕集),ITKC_MO_0001A_0020_010_0020,ITKC_MO_0001A_0020_010_0020_2003_A001_XML,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等\n廻。雲南通和兼進獻國信金銀器物疋段香藥馬等\...
...,...,...,...,...,...
653262,ITKC_MO_1259A,1259. 소눌집(小訥集),ITKC_MO_1259A_0480_020_0030,ITKC_MO_1259A_0480_020_0030_2021_B150_XML,烈婦李氏。星山君軾之後也。年十七嫁順天朴泰東。\n明年夫死。手製襲斂具。葬若祥盡禮。堂有姑。...
653263,ITKC_MO_1259A,1259. 소눌집(小訥集),ITKC_MO_1259A_0480_020_0040,ITKC_MO_1259A_0480_020_0040_2021_B150_XML,具一勞名泰祚字幸文。昌原人。常曰我勞一家子弟\n視我勞。不敢自逸。我逸視我逸。不肯自勞。對家...
653264,ITKC_MO_1259A,1259. 소눌집(小訥集),ITKC_MO_1259A_0480_020_0050,ITKC_MO_1259A_0480_020_0050_2021_B150_XML,朴琮弼者。密城人。前昌寧郡守尙鎰之子。兄弟凡七\n人。序居第一。今年三十。第四弟琮碩年十三。...
653265,ITKC_MO_1259A,1259. 소눌집(小訥集),ITKC_MO_1259A_0480_020_0060,ITKC_MO_1259A_0480_020_0060_2021_B150_XML,烈婦尹孺人。坡平世家士人持復女。晉陽鄭伯奎之\n妻也。在家以善事父母。爲宗族所稱。及嫁得舅姑...


In [ ]:
# 중복확인
df_explode.drop_duplicates(subset=['id','dci','content'])

,bib,book,id,dci,content
0,ITKC_MO_0001A,1. 계원필경집(桂苑筆耕集),ITKC_MO_0001A_0010_010_0010,ITKC_MO_0001A_0010_010_0010_2003_A001_XML,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本\n也。黼黻文繡之美。而疏布之尙。反女功之始也...
1,ITKC_MO_0001A,1. 계원필경집(桂苑筆耕集),ITKC_MO_0001A_0010_010_0020,ITKC_MO_0001A_0010_010_0020_2003_A001_XML,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府\n時公私應酬之作。而東還之後。手編表進于朝者也\...
2,ITKC_MO_0001A,1. 계원필경집(桂苑筆耕集),ITKC_MO_0001A_0010_010_0030,ITKC_MO_0001A_0010_010_0030_2003_A001_XML,淮南入本國兼送詔書等使前都統巡官承務郎\n侍御史內供奉賜紫金魚袋臣崔致遠進所著雜\n詩賦及表...
3,ITKC_MO_0001A,1. 계원필경집(桂苑筆耕集),ITKC_MO_0001A_0020_010_0010,ITKC_MO_0001A_0020_010_0010_2003_A001_XML,臣某言。今月某日。得進奏院狀報。奉十一日宣下改\n廣明元年爲中和元年者。展義龜城。易名鳳紀。...
4,ITKC_MO_0001A,1. 계원필경집(桂苑筆耕集),ITKC_MO_0001A_0020_010_0020,ITKC_MO_0001A_0020_010_0020_2003_A001_XML,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等\n廻。雲南通和兼進獻國信金銀器物疋段香藥馬等\...
...,...,...,...,...,...
653262,ITKC_MO_1259A,1259. 소눌집(小訥集),ITKC_MO_1259A_0480_020_0030,ITKC_MO_1259A_0480_020_0030_2021_B150_XML,烈婦李氏。星山君軾之後也。年十七嫁順天朴泰東。\n明年夫死。手製襲斂具。葬若祥盡禮。堂有姑。...
653263,ITKC_MO_1259A,1259. 소눌집(小訥集),ITKC_MO_1259A_0480_020_0040,ITKC_MO_1259A_0480_020_0040_2021_B150_XML,具一勞名泰祚字幸文。昌原人。常曰我勞一家子弟\n視我勞。不敢自逸。我逸視我逸。不肯自勞。對家...
653264,ITKC_MO_1259A,1259. 소눌집(小訥集),ITKC_MO_1259A_0480_020_0050,ITKC_MO_1259A_0480_020_0050_2021_B150_XML,朴琮弼者。密城人。前昌寧郡守尙鎰之子。兄弟凡七\n人。序居第一。今年三十。第四弟琮碩年十三。...
653265,ITKC_MO_1259A,1259. 소눌집(小訥集),ITKC_MO_1259A_0480_020_0060,ITKC_MO_1259A_0480_020_0060_2021_B150_XML,烈婦尹孺人。坡平世家士人持復女。晉陽鄭伯奎之\n妻也。在家以善事父母。爲宗族所稱。及嫁得舅姑...


In [ ]:
# 엑셀과 CSV 파일로 전체 저장
# df_explode.to_excel('./한국문집총간_기본정보.xlsx',index=None) # 엑셀로 저장하는 것 비추천. 수십만 줄이기 떄문
# df_explode.to_csv('./한국문집총간_기본정보.csv',index=None) # Emeditor 같은 대용량 텍스트를 읽어내는 텍스트 에디터에서 csv 형태로 읽어내길 추천

In [ ]:
# 계원필경집만 저장
df_explode.loc[df_explode['book']=='1. 계원필경집(桂苑筆耕集)'].to_excel('./1.계원필경집.xlsx')

In [ ]:
# 본문에 특정 단어가 들어간 문서 찾기
df_explode.loc[df_explode['content'].str.contains('仁')]

##### 연계 정보

In [ ]:
df_asso = pd.DataFrame(row_asso_list)
df_asso

,id,type,연계시작,연계종료
0,ITKC_MO_0001A_0010_010_0010,이미지,ITKC_MO_0001A_A001_003A,ITKC_MO_0001A_A001_003D
1,ITKC_MO_0001A_0010_010_0010,번역문,ITKC_BT_0001A_0010_000_0010,ITKC_BT_0001A_0010_000_0010
2,ITKC_MO_0001A_0010_010_0020,이미지,ITKC_MO_0001A_A001_004A,ITKC_MO_0001A_A001_005A
3,ITKC_MO_0001A_0010_010_0020,번역문,ITKC_BT_0001A_0010_000_0020,ITKC_BT_0001A_0010_000_0020
4,ITKC_MO_0001A_0010_010_0030,이미지,ITKC_MO_0001A_A001_005C,ITKC_MO_0001A_A001_006A
...,...,...,...,...
800489,ITKC_MO_1259A_0480_020_0030,이미지,ITKC_MO_1259A_B150_685C,ITKC_MO_1259A_B150_686A
800490,ITKC_MO_1259A_0480_020_0040,이미지,ITKC_MO_1259A_B150_686A,ITKC_MO_1259A_B150_687C
800491,ITKC_MO_1259A_0480_020_0050,이미지,ITKC_MO_1259A_B150_687C,ITKC_MO_1259A_B150_688A
800492,ITKC_MO_1259A_0480_020_0060,이미지,ITKC_MO_1259A_B150_688A,ITKC_MO_1259A_B150_688C


In [ ]:
# 중복 확인
df_asso = df_asso.drop_duplicates().reset_index(drop=True)
df_asso

,id,type,연계시작,연계종료
0,ITKC_MO_0001A_0010_010_0010,이미지,ITKC_MO_0001A_A001_003A,ITKC_MO_0001A_A001_003D
1,ITKC_MO_0001A_0010_010_0010,번역문,ITKC_BT_0001A_0010_000_0010,ITKC_BT_0001A_0010_000_0010
2,ITKC_MO_0001A_0010_010_0020,이미지,ITKC_MO_0001A_A001_004A,ITKC_MO_0001A_A001_005A
3,ITKC_MO_0001A_0010_010_0020,번역문,ITKC_BT_0001A_0010_000_0020,ITKC_BT_0001A_0010_000_0020
4,ITKC_MO_0001A_0010_010_0030,이미지,ITKC_MO_0001A_A001_005C,ITKC_MO_0001A_A001_006A
...,...,...,...,...
800489,ITKC_MO_1259A_0480_020_0030,이미지,ITKC_MO_1259A_B150_685C,ITKC_MO_1259A_B150_686A
800490,ITKC_MO_1259A_0480_020_0040,이미지,ITKC_MO_1259A_B150_686A,ITKC_MO_1259A_B150_687C
800491,ITKC_MO_1259A_0480_020_0050,이미지,ITKC_MO_1259A_B150_687C,ITKC_MO_1259A_B150_688A
800492,ITKC_MO_1259A_0480_020_0060,이미지,ITKC_MO_1259A_B150_688A,ITKC_MO_1259A_B150_688C


In [ ]:
# 연계 type 빈도 확인
df_asso['type'].value_counts()

이미지     653267
번역문     105624
교감표점     41603
Name: type, dtype: int64

In [ ]:
# 엑셀과 CSV 파일로 저장 (오래걸림)
# df_asso.to_excel('./한국문집총간_연계정보.xlsx')
# df_asso.to_csv('./한국문집총간_연계정보.csv')